# Identifying and understanding **KPIs**


### This mobile app that offers meditation services for a paid subscription as well as one-off in-app purchases. The app is growing quickly and we are motivated to maintain a strong free-trial to paying user conversion rate. Additionally, we want to maintain strength in a variety of other business areas as we will see. 

In [14]:
import pandas as pd
customer_demographics = pd.read_csv('datasets/customer_demographics.csv')
customer_demographics 
print(customer_demographics.head())

          uid              reg_date device gender country  age
0  54030035.0  2017-06-29T00:00:00Z    and      M     USA   19
1  72574201.0  2018-03-05T00:00:00Z    iOS      F     TUR   22
2  64187558.0  2016-02-07T00:00:00Z    iOS      M     USA   16
3  92513925.0  2017-05-25T00:00:00Z    and      M     BRA   41
4  99231338.0  2017-03-26T00:00:00Z    iOS      M     FRA   59


In [15]:
import pandas as pd
# Load customer_subscriptions
customer_subscriptions  = pd.read_csv('customer_subscriptions.csv')
# Print the head of customer_subscriptions 
print(customer_subscriptions.head())

         date       uid            sku  price
0  2017-07-10  41195147  sku_three_499    499
1  2017-07-15  41195147  sku_three_499    499
2  2017-11-12  41195147   sku_four_599    599
3  2017-09-26  91591874    sku_two_299    299
4  2017-12-01  91591874   sku_four_599    599


###  One question in defining our KPI is over what interval should we consider the conversion rate? The conversion immediately after lapse? one week after? One month? One way to decide this is to see the generalizability of these statistics across different demographic groups. Stability in this way is desired so we don't need custom KPIs for each breakdown. A second is to see if one is more correlated with important factors like retention or spending than the others.

## Merging Mechanics
### pandas merge() method. This performs the equivalent of a SQL join on two dataFrames. There are two ways to call this method, either as a method of pandas or as a method of a dataFrame object. 
### As in SQL we have a left and right table. We call the merge method on one of our dataFrames, and this is considered the left dataFrame. In this case our demographics dataset. Next, we specify the right dataFrame as our first argument, in this case the subscription data. Then, we specify the `how` argument. This can be one of four values: inner, outer, left, or right, each analogous to a sql join. Understanding SQL is not important for this course, it suffices to say that these arguments specify the behavior of which rows are returned in the final output. For our purpose we will use an inner join which returns all rows that are matched between the two dataFrames. The next argument is the `on` argument. This is a list of fields, that appear in both dataFrames, which we want to match the rows on. There is a way to specify this argument when the columns differ in name, but we will not cover that here. We will match on the `uid`. As we can see in the output, the rows are associated with a corresponding row from the other dataFrame.

In [16]:
# merge customer_demographics (left) and customer_subscriptions (right)
sub_data_demo = customer_demographics.merge(
                                     # right dataframe
                                     customer_subscriptions,
                                     # join type
                                     how = 'inner',
                                     # columns to match
                                     on = ['uid'])
sub_data_demo.head()

,uid,reg_date,device,gender,country,age,date,sku,price
0,92513925.0,2017-05-25T00:00:00Z,and,M,BRA,41,2017-10-20,sku_three_499,499
1,92513925.0,2017-05-25T00:00:00Z,and,M,BRA,41,2017-05-29,sku_two_299,299
2,92513925.0,2017-05-25T00:00:00Z,and,M,BRA,41,2017-08-23,sku_four_599,599
3,92513925.0,2017-05-25T00:00:00Z,and,M,BRA,41,2018-03-26,sku_six_1299,299
4,16377492.0,2016-10-16T00:00:00Z,and,M,BRA,20,2018-03-17,sku_one_199,199


## Exploratory analysis of KPIs

In [20]:
# sub_data_demo - combined demographics and purchase date
sub_data_grp = sub_data_demo.groupby(by=['country', 'device'],
                                    axis=0,
                                    as_index=False)
sub_data_grp

## Aggregating data - mean price paid per group


In [21]:
# mean price paid for each country/device
sub_data_grp.price.mean()

,country,device,price
0,BRA,and,412.985594
1,BRA,iOS,404.739300
2,CAN,and,406.826087
3,CAN,iOS,386.573964
4,DEU,and,402.474903
5,DEU,iOS,417.639798
6,FRA,and,418.377163
7,FRA,iOS,382.921569
8,TUR,and,433.913793
9,TUR,iOS,390.176471


## Aggregate data : .agg()

In [23]:
# find the mean price paid with agg
sub_data_grp.price.agg('mean').head()

,country,device,price
0,BRA,and,412.985594
1,BRA,iOS,404.739300
2,CAN,and,406.826087
3,CAN,iOS,386.573964
4,DEU,and,402.474903


## .agg():multiple functions, multiple columns
Pass a dictionary of column names and aggregation functions 

In [24]:
# Calculate multiple metrics across different groups
sub_data_grp.agg({'price' : ['mean', 'min', 'max'],
                            'age' :['mean', 'min', 'max']})

country device       price                 age        
                         mean min  max       mean min max
0      BRA    and  412.985594  99  899  23.913565  15  62
1      BRA    iOS  404.739300  99  899  23.975681  15  66
2      CAN    and  406.826087  99  899  23.282609  15  49
3      CAN    iOS  386.573964  99  899  25.023669  15  57
4      DEU    and  402.474903  99  899  23.814672  15  66
5      DEU    iOS  417.639798  99  899  20.773300  15  42
6      FRA    and  418.377163  99  899  23.200692  15  55
7      FRA    iOS  382.921569  99  899  24.407843  15  59
8      TUR    and  433.913793  99  899  24.004310  15  56
9      TUR    iOS  390.176471  99  899  22.591176  15  51
10     USA    and  399.000000  99  899  24.564738  15  65
11     USA    iOS  410.684783  99  899  24.144022  15  75

## .agg(): custom functions 

In [27]:
def truncated_mean(data):
    ##Compute the mean excluding outliers
    top_val = data.quantile(.9)
    bot_val = data.quantile(.1)
    trunc_data = data[(data <= top_val) & (data >= bot_val)]
    mean = trunc_data.mean()
    return(mean)

In [28]:
# Find the truncated mean age by group
sub_data_grp.agg({'age': [truncated_mean]})

,country,device,age
,,,truncated_mean
0,BRA,and,22.798977
1,BRA,iOS,22.784946
2,CAN,and,23.335000
3,CAN,iOS,24.048780
4,DEU,and,21.640257
5,DEU,iOS,19.611413
6,FRA,and,21.786885
7,FRA,iOS,23.123457
8,TUR,and,22.422572


##  Conversion rate: maximum lapse date

In [1]:
import pandas as pd
from datetime import datetime, timedelta

current_date = pd.to_datetime('2018-03-17')

## Lapse date: Date the trial ends for given user

In [ ]:
# What is the maximum lapse date in our data
print(sub_data.lapse_date.max())

## KPI calculation: restrict users by lapse date

In [ ]:
# latest lapse date a before today
from datetime import datetime, timedelta
max_lapse_date = datetime.now() - timedelta(days=7)
# restrict to users lapsed before max_lapse_date
conv_sub_data = sub_data_demo[(sub_data_demo.lapse_date < max_lapse_date)]
# count the users remaining in our data
total_users_count = conv_sub_data.price.count()
print(total_users_count)

In [ ]:
# Latest subscription date:within 7 days od lapsing
max_sub_date = conv_sub_data.lapse_date + timedelta(days=7)



# Filter the users with non-zero subscription price who subscribed before max_sub_date
total_subs = conv_sub_data[
    (conv_sub_data.price > 0)&
    (conv_sub_data.subscription_date <= max_sub_date)]

# Count the users remaining in our data
total_subs_count = total_subs.price.count()
print(total_subs_count)

## conversion rate = Total subscribers / Potential subscribers 

In [ ]:
# calculate the conversion rate with our previous values
conversion_rate = total_subs_count / total_users_count
print(conversion_rate)

## Cohort conversion rate

In [ ]:
# Create a copy of our dataframe
conv_sub_data = conv_sub_data.copy()

# keep users who lapsed prior to the last 14 days (2 week)
max_lapse_date = current_date - timedelta(days=14)
conv_sub_delta = sub_data_demo[(sub_data_demo.lapse_date <= max_lapse_date)]

# find the days between lapse and subscription if they subscribed... and pd.NaT otherwise
sub_time = np.where(
              # if: a subscription data exist
               conv_sub_data.subscription_date.notnull(),
              # then:find how many days since their lapse
                (conv_sub_data.subscription _date - conv_sub_data.lapse_date).dt.days, 
              #else: set the value to p.NaT
               pd.NaT)

# create a new column 'sub_time'
conv_sub_data['sub_time'] = sub_time 

### grc7(), grc14(): calculate the 7 and 14 day conversion rates

In [ ]:
# group by the relevant cohorts
purchase_cohorts = data.groupby(by=['gender', 'device'], as_index=False)

# finf the conversion rate for each cohort using gcr7, gcr14
purchase_cohorts.agg({sub_time: [gcr7, gcr14]})

## How to choose KPI metrics?
To conclude our discussion on KPIs, it is important to note that while there an infinite number of KPIs we want to choose carefully which to rely on. One factor in determining this is how long it takes to gain insight on a metric. To find the monthly conversion rate, we would need to wait a month from the lapse date. This can make it impractical to monitor on an actionable time scale. Other ways to uncover KPIs include exploratory analysis which can reveal relationships between metrics and key results. Additionally, these metrics can be tied to the business metrics in important ways.